# Additional materials
This file contains the source code, comments to it and instructions for building.
It is also assumed, that the user already has [python](https://www.python.org/downloads/) downloaded and [pip](https://pip.pypa.io/en/stable/installation/) installed. If you are using other package managers such as Homebrew or Conda, you'll have to ensure that all used libraries are installed on your device

## Getting and processing data for the graph
As a result of human imperfection, there was a need to process data automatically. This removes the human factor and make the work way more easier. However, the creation of such a system can be very laborious, long-drawn-out, requiring patience and, in total, ungrateful. But in all we can get a very pretty result))

### Installing libraries
I used 3 libraries to handle, parse and process data, respectively, [Requests](https://pypi.org/project/requests/), [Beautiful Soup](https://pypi.org/project/beautifulsoup4/) and [Pandas](https://pypi.org/project/pandas/)

**update:** To parse the HTML code in Beatifoul Soup, [lxml](https://pypi.org/project/lxml/) is needed.

To construct and compute my graph I used [Networkx](https://pypi.org/project/networkx/) – a tool for working with complex networks, which is very helpful when we have many nodes and edges in our graph.

Finally, to export my graph into my $\rm\TeX$ cleanup, I used [tikzplotlib](https://pypi.org/project/tikzplotlib/). It will convert my graph object to a PGFPlot figure.

In [ ]:
%pip install requests
%pip install beautifulsoup4
%pip install pandas
%pip install lxml
#
%pip install networkx
#
%pip install tikzplotlib

### Importing libraries
Is there something to explain? :)

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#
import networkx as nx
#
import tikzplotlib

### Getting countries
It will be fabulous to have a list of land borders of European countries on Wiki in one, but they are separated, so the hard way you choose :(

We will first send a request to the [page](https://simple.wikipedia.org/wiki/List_of_European_countries) with the table of all European countries and check the status code (not necessary for this case, but useful in general):

In [3]:
r_countries = requests.get(
    'https://simple.wikipedia.org/wiki/List_of_European_countries')
if r_countries.status_code != 200:  # means that the scrapping is not legally possible
    print("Error: website doesn't allow to download the source code")
    quit

We'll create a Beautiful Soup object by reading and parsing the text from our request and then find the part with the table in it.
Thereafter we have to delete (decompose) all data under tag 'sup' which stands for superscript which is used for footnotes in Wikipedia to "clear" countries name:

In [4]:
soup_countries = BeautifulSoup(r_countries.text, 'html.parser')
table_countries = soup_countries.find('table', {'class': "wikitable"})
for data in table_countries(['sup']):
    data.decompose()

Finally we can build our dataframe. First we read the info from our table into a list of DataFrame objects and then build the DataFrame itself.
Then I deleted some columns I don't need and rename the column with countries to "Country". At last, I sorted the countries and reset the indexes:

In [5]:
df_countries = pd.read_html(str(table_countries))
df_countries = pd.DataFrame(df_countries[0])
df_countries = df_countries.drop(df_countries.columns[[1, 2, 3]], axis=1)
df_countries.columns = ['Country', 'Capital']
df_countries = df_countries.sort_values(by=['Country'])
df_countries = df_countries.reset_index(drop=True)


You can easily check your DataFrame just by printing the object or using the method "head" to print the n first rows:

In [20]:
print(df_countries)

                   Country           Capital
0                  Albania            Tirana
1                  Andorra  Andorra la Vella
2                  Armenia           Yerevan
3                  Austria            Vienna
4                  Belarus             Minsk
5                  Belgium          Brussels
6   Bosnia and Herzegovina          Sarajevo
7                 Bulgaria             Sofia
8                  Croatia            Zagreb
9                   Cyprus           Nicosia
10          Czech Republic            Prague
11                 Denmark        Copenhagen
12                 Estonia           Tallinn
13                 Finland          Helsinki
14                  France             Paris
15                 Georgia           Tbilisi
16                 Germany            Berlin
17                  Greece            Athens
18                 Hungary          Budapest
19                 Iceland         Reykjavík
20                 Ireland            Dublin
21        

### Getting borders
We'll get the data from the [table](https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_land_borders) with all countries and their land borders and clear it (country and borders remain)

In [7]:
r_borders = requests.get(
    'https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_land_borders')
if r_borders.status_code != 200:
    print("Error: website doesn't allow to download the source code")
    quit

soup_borders = BeautifulSoup(r_borders.text, 'html.parser')
table_borders = soup_borders.find('table', {'class': "wikitable"})
for data in table_borders(['sup']):
    data.decompose()
df_borders = pd.read_html(str(table_borders))
df_borders = pd.DataFrame(df_borders[0])
df_borders = df_borders.droplevel(0, axis=1)
df_borders = df_borders.drop(df_borders.columns[[1, 2, 3, 4]], axis=1)
df_borders.columns = ['Country', 'Borders']

In [18]:
df_borders.at[47, 'Country'] = 'Denmark'
df_borders.at[63, 'Country'] = 'France'
df_borders.at[131, 'Country'] = 'Netherlands'
df_borders.at[194, 'Country'] = 'United Kingdom'
df_borders.at[45, 'Borders'] = float("NaN")
type(df_borders.at[32, 'Borders'])
#df_borders.loc[30:60, :]

float

In [21]:
df = df_countries.merge(df_borders, how='left')
df['Borders'] = df['Borders'].str.split('\n')
#df[['Borders']]
df

,Country,Capital,Borders
0,Albania,Tirana,[Greece: 282 km (175 mi) Kosovo: 112 km (70 mi...
1,Andorra,Andorra la Vella,[France: 56.6 km (35.2 mi) Spain: 63.7 km (39....
2,Armenia,Yerevan,[Azerbaijan (6): 787 km (489 mi) Georgia: 164 ...
3,Austria,Vienna,[Czech Republic: 362 km (225 mi) Germany: 784 ...
4,Belarus,Minsk,[Latvia: 141 km (88 mi) Lithuania: 502 km (312...
5,Belgium,Brussels,[France: 620 km (390 mi) Germany (6): 167 km (...
6,Bosnia and Herzegovina,Sarajevo,[Croatia (2): 932 km (579 mi) Montenegro: 225 ...
7,Bulgaria,Sofia,[Greece: 494 km (307 mi) North Macedonia: 148 ...
8,Croatia,Zagreb,[Bosnia and Herzegovina (2): 932 km (579 mi) H...
9,Cyprus,Nicosia,NaN
